In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.applications import inception_v3
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Model 
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

%matplotlib inline

# Set default matplotlib params
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

c:\users\tomforge\anaconda3\envs\aicv\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
TRAIN_DIR = 'data/train'
TEST_DIR = 'data/test'
NUM_TRAIN, NUM_TEST = 34398, 3813
# Use ImageNet image sizes
IMG_WIDTH, IMG_HEIGHT = 256, 256
FC_SIZE = 1024
NUM_EPOCHS = 25
BATCH_SIZE = 182
NB_CLASSES = 18

In [7]:
#Load the Inception_V3 model, leave out final fully connected layer, add globalavg pooling
inception_model = inception_v3.InceptionV3(weights='imagenet', include_top=False, pooling='avg',\
                                           input_shape = (IMG_WIDTH, IMG_HEIGHT, 3))
# Freeze the layers
for layer in inception_model.layers:
    layer.trainable = False
    
# Add custom top layer
x = inception_model.output
x = Dense(FC_SIZE, activation='relu')(x) # Fully-connected layer, relu activation
predictions = Dense(NB_CLASSES, activation='softmax')(x) # Channel into NB_CLASSES cells

final_model = Model(inputs=inception_model.input, outputs=predictions)

# Compile the model for training
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Save the model according to the conditions
checkpoint = ModelCheckpoint("inceptionv3.h5", monitor='val_acc', verbose=1, save_best_only=True,
                             save_weights_only=False, mode='auto', period=1)
# Terminate early if validation accuracy stop improving over 10 epochs
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

In [8]:
# Initiate the train and test generators with data augumentation 
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    fill_mode = "nearest",
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3,
    rotation_range=30)

# For test data, rescaling will do
test_datagen = ImageDataGenerator(
    rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    batch_size = BATCH_SIZE, 
    class_mode = "categorical")

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    class_mode = "categorical")

Found 34398 images belonging to 18 classes.
Found 3813 images belonging to 18 classes.


In [9]:
# Train the model 
final_model.fit_generator(
    train_generator,
    steps_per_epoch = NUM_TRAIN/BATCH_SIZE,
    epochs = NUM_EPOCHS,
    validation_data = test_generator,
    callbacks = [checkpoint, early])

Epoch 1/25
189/189 [==============================] - 1137s 6s/step - loss: 1.5985 - acc: 0.4778 - val_loss: 1.3496 - val_acc: 0.5872

Epoch 00001: val_acc improved from -inf to 0.58720, saving model to inceptionv3.h5
Epoch 2/25
189/189 [==============================] - 1100s 6s/step - loss: 1.3181 - acc: 0.5538 - val_loss: 1.2695 - val_acc: 0.5982

Epoch 00002: val_acc improved from 0.58720 to 0.59822, saving model to inceptionv3.h5
Epoch 3/25
189/189 [==============================] - 592s 3s/step - loss: 1.2489 - acc: 0.5759 - val_loss: 1.2987 - val_acc: 0.5945

Epoch 00003: val_acc did not improve
Epoch 4/25
189/189 [==============================] - 592s 3s/step - loss: 1.2309 - acc: 0.5807 - val_loss: 1.2323 - val_acc: 0.6040

Epoch 00004: val_acc improved from 0.59822 to 0.60399, saving model to inceptionv3.h5
Epoch 5/25
189/189 [==============================] - 682s 4s/step - loss: 1.2035 - acc: 0.5878 - val_loss: 1.2534 - val_acc: 0.6179

Epoch 00005: val_acc improved from 0

KeyboardInterrupt: 